In [1]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.4 MB/s eta 0:00:00:00:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 5.6 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=1cafcba79223cde9f8be96659fee4138af1c6828d5f71a45a9887d148e55a003
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7


In [3]:
import whisper
import os
import random
import torch
import librosa
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
from easyocr import Reader
from transformers import WhisperProcessor
from torch.utils.data import DataLoader
from torch import nn
import re
import soundfile as sf
from jiwer import wer, cer  
from tqdm import tqdm 

# Dataset Path

In [4]:
# Directories
audio_dir = '/kaggle/input/captcha-dataset/captchas/audio'
images_dir = '/kaggle/input/captcha-dataset/captchas/images'

audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.wav')]
image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]

print(f"Total audio files: {len(audio_files)}")
print(f"Total image files: {len(image_files)}")

Total audio files: 10000
Total image files: 10000


# Implementation of Whisper AI(small) and OCR to generate text for 10 random samples

In [21]:
audio_dir = '/kaggle/input/captcha-dataset/captchas/audio'
image_dir = '/kaggle/input/captcha-dataset/captchas/images'

# Initialize EasyOCR and Whisper
ocr_reader = Reader(['en'])  # EasyOCR setup
whisper_model = whisper.load_model("base")  # Whisper model

# Dictionary to convert number words to digits
number_map = {
    "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
    "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9"
}

# Dictionary for common misheard words
misheard_map = {
    "mall": "small", "moll": "small", "capitun": "capital", "capitan": "capital",
    "apital": "capital", "capitole": "capital", "zimro": "0", "smaller": "a"
}

# Step 1: Replace number words with digits
def replace_number_words(text):
    for word, digit in number_map.items():
        text = re.sub(rf"\b{word}\b", digit, text, flags=re.IGNORECASE)
    return text

# Step 2: Fix common misheard words
def fix_misheard_words(text):
    for wrong, correct in misheard_map.items():
        text = re.sub(rf"\b{wrong}\b", correct, text, flags=re.IGNORECASE)
    return text

# Step 3: Replace "for" with "4"
def replace_for_with_4(text):
    text = re.sub(r"\bfor\b", "4", text, flags=re.IGNORECASE)
    return text

# Step 4: Process capitalization and handle "edge" replacement
def process_capitalization(text):
    matches = re.findall(r"(capital|small) (\w+)", text, re.IGNORECASE)
    cleaned_text = ""

    i = 0
    while i < len(text):
        match_found = False
        
        for match in matches:
            marker, letter = match
            marker_index = text.lower().find(f"{marker.lower()} {letter.lower()}")
            
            if marker_index == i:
                if marker.lower() == "capital":
                    cleaned_text += letter.upper()
                else:
                    cleaned_text += letter.lower()
                i += len(marker) + 2
                match_found = True
                break

        if not match_found:
            cleaned_text += text[i]
            i += 1

    # Handle "edge" replacement based on the previous marker
    cleaned_text = re.sub(r"\bsmall\s+edge\b", "h", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"\bcapital\s+edge\b", "H", cleaned_text, flags=re.IGNORECASE)

    # Remove remaining "capital" or "small" words
    cleaned_text = re.sub(r"\b(capital|small)\b", "", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"[^a-zA-Z0-9]", "", cleaned_text)  # Keep only A-Z, a-z, 0-9
    return cleaned_text

# Function to get Whisper transcription
def get_whisper_transcription(audio_file):
    text = ""
    
    # Try Whisper transcription up to 3 times if length is < 6
    for _ in range(3):
        result = whisper_model.transcribe(audio_file)
        text = result["text"]
        text = replace_number_words(text)  # Replace number words with digits
        text = fix_misheard_words(text)  # Fix misheard words
        text = replace_for_with_4(text)  # Replace "for" with "4"
        text = process_capitalization(text)  # Process capitalization & "edge"
        
        if len(text) >= 6:  # Ensure minimum length of 6
            break

    return text[:6]  # Ensure the output is exactly 6 characters

# Function to get text from image using EasyOCR
def get_image_text(image_file):
    ocr_result = ocr_reader.readtext(image_file)
    text = ''.join([res[1] for res in ocr_result])
    return text

def process_random_samples(audio_dir, image_dir, num_samples=10):
    audio_files = random.sample(os.listdir(audio_dir), num_samples)
    
    for audio_file in audio_files:
        audio_path = os.path.join(audio_dir, audio_file)
        image_file = audio_file.replace('.wav', '.png')  # Assuming same filename for audio and image
        image_path = os.path.join(image_dir, image_file)

        whisper_text = get_whisper_transcription(audio_path)
        ground_truth_text = get_image_text(image_path)

        print(f"Audio file: {audio_file}")
        print(f"Whisper AI Transcription: {whisper_text}")
        print(f"Ground Truth (OCR): {ground_truth_text}")
        print("="*50)

# Run the function to process random samples
process_random_samples(audio_dir, image_dir, num_samples=10)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Audio file: captcha_2800.wav
Whisper AI Transcription: P0h1kQ
Ground Truth (OCR): PohzkQ
Audio file: captcha_9383.wav
Whisper AI Transcription: pK2Udc
Ground Truth (OCR): pKZUdc
Audio file: captcha_0862.wav
Whisper AI Transcription: mVv2N
Ground Truth (OCR): mBZv2N
Audio file: captcha_2692.wav
Whisper AI Transcription: F4aE13
Ground Truth (OCR): FaaE13
Audio file: captcha_5985.wav
Whisper AI Transcription: ASVKVA
Ground Truth (OCR): AsVKBE
Audio file: captcha_8862.wav
Whisper AI Transcription: apGv5v
Ground Truth (OCR): epGvsb
Audio file: captcha_2196.wav
Whisper AI Transcription: LHlzs7
Ground Truth (OCR): LHlzs7
Audio file: captcha_0318.wav
Whisper AI Transcription: PHW1Qb
Ground Truth (OCR): PHWIQb
Audio file: captcha_3554.wav
Whisper AI Transcription: wi8ciu
Ground Truth (OCR): wi8ciu
Audio file: captcha_0087.wav
Whisper AI Transcription: auIGL0
Ground Truth (OCR): aulGLO


# Whisper AI(small) Matching Accuracy Scores

In [28]:
audio_dir = '/kaggle/input/captcha-dataset/captchas/audio'
image_dir = '/kaggle/input/captcha-dataset/captchas/images'

ocr_reader = Reader(['en'])  # EasyOCR setup
whisper_model = whisper.load_model("base")  # Whisper model

# Dictionary to convert number words to digits
number_map = {
    "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
    "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9"
}

# Dictionary for common misheard words
misheard_map = {
    "mall": "small", "moll": "small", "capitun": "capital", "capitan": "capital",
    "apital": "capital", "capitole": "capital", "zimro": "0", "smaller": "a"
}

# Step 1: Replace number words with digits
def replace_number_words(text):
    for word, digit in number_map.items():
        text = re.sub(rf"\b{word}\b", digit, text, flags=re.IGNORECASE)
    return text

# Step 2: Fix common misheard words
def fix_misheard_words(text):
    for wrong, correct in misheard_map.items():
        text = re.sub(rf"\b{wrong}\b", correct, text, flags=re.IGNORECASE)
    return text

# Step 3: Replace "for" with "4"
def replace_for_with_4(text):
    text = re.sub(r"\bfor\b", "4", text, flags=re.IGNORECASE)
    return text

# Step 4: Process capitalization and handle "edge" replacement
def process_capitalization(text):
    matches = re.findall(r"(capital|small) (\w+)", text, re.IGNORECASE)
    cleaned_text = ""

    i = 0
    while i < len(text):
        match_found = False
        
        for match in matches:
            marker, letter = match
            marker_index = text.lower().find(f"{marker.lower()} {letter.lower()}")
            
            if marker_index == i:
                if marker.lower() == "capital":
                    cleaned_text += letter.upper()
                else:
                    cleaned_text += letter.lower()
                i += len(marker) + 2
                match_found = True
                break

        if not match_found:
            cleaned_text += text[i]
            i += 1

    # Handle "edge" replacement based on the previous marker
    cleaned_text = re.sub(r"\bsmall\s+edge\b", "h", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"\bcapital\s+edge\b", "H", cleaned_text, flags=re.IGNORECASE)

    # Remove remaining "capital" or "small" words
    cleaned_text = re.sub(r"\b(capital|small)\b", "", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"[^a-zA-Z0-9]", "", cleaned_text)  # Keep only A-Z, a-z, 0-9
    return cleaned_text

# Function to get Whisper transcription
def get_whisper_transcription(audio_file):
    text = ""
    
    # Try Whisper transcription up to 3 times if length is < 6
    for _ in range(3):
        result = whisper_model.transcribe(audio_file)
        text = result["text"]
        text = replace_number_words(text)  # Replace number words with digits
        text = fix_misheard_words(text)  # Fix misheard words
        text = replace_for_with_4(text)  # Replace "for" with "4"
        text = process_capitalization(text)  # Process capitalization & "edge"
        
        if len(text) >= 6:  # Ensure minimum length of 6
            break

    return text[:6]  # Ensure the output is exactly 6 characters

def get_image_text(image_file):
    ocr_result = ocr_reader.readtext(image_file)
    text = ''.join([res[1] for res in ocr_result])
    return text

def process_all_samples(audio_dir, image_dir):
    all_files = os.listdir(audio_dir)
    total_files = len(all_files)
    cer_scores = []
    
    for i, audio_file in enumerate(tqdm(all_files, desc="Processing")):
        audio_path = os.path.join(audio_dir, audio_file)
        image_file = audio_file.replace('.wav', '.png')  # Assuming matching names
        image_path = os.path.join(image_dir, image_file)

        whisper_text = get_whisper_transcription(audio_path)

        ground_truth_text = get_image_text(image_path)

        if not ground_truth_text or not whisper_text:
            continue 

        # Compute CER (Character Error Rate)
        error_rate = cer(ground_truth_text, whisper_text)  # CER calculation
        accuracy = max(0, 100 - (error_rate * 100))  
        cer_scores.append(accuracy)

        # Print progress every 500 samples
        if (i + 1) % 500 == 0:
            avg_accuracy = sum(cer_scores) / len(cer_scores)
            print(f"\nProgress: {i+1}/{total_files} samples processed. Current Accuracy: {avg_accuracy:.2f}%\n")

    # Final average accuracy
    avg_accuracy = sum(cer_scores) / len(cer_scores) if cer_scores else 0
    print(f"\nFinal Accuracy after processing {total_files} samples: {avg_accuracy:.2f}%\n")

# Run the function to process all samples
process_all_samples(audio_dir, image_dir)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
Processing:  


Progress: 1000/10000 samples processed. Current Accuracy: 72.88%



Processing:  15%|█▌        | 1500/10000 [11:10<57:26,  2.47it/s]  


Progress: 1500/10000 samples processed. Current Accuracy: 73.08%



Processing:  20%|██        | 2000/10000 [15:20<56:21,  2.37it/s]   

...


Progress: 9000/10000 samples processed. Current Accuracy: 72.99%



Processing:  95%|█████████▌| 9500/10000 [1:13:19<03:09,  2.64it/s]


Progress: 9500/10000 samples processed. Current Accuracy: 72.98%



Processing: 100%|██████████| 10000/10000 [1:17:04<00:00,  2.16it/s]


Progress: 10000/10000 samples processed. Current Accuracy: 73.07%


Final Accuracy after processing 10000 samples: 73.07%



# Whisper AI(small) - No. of texts matched

In [30]:
audio_dir = '/kaggle/input/captcha-dataset/captchas/audio'
image_dir = '/kaggle/input/captcha-dataset/captchas/images'

ocr_reader = Reader(['en'])  # EasyOCR setup
whisper_model = whisper.load_model("base")  # Whisper model

# Dictionary to convert number words to digits
number_map = {
    "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
    "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9"
}

# Dictionary for common misheard words
misheard_map = {
    "mall": "small", "moll": "small", "capitun": "capital", "capitan": "capital",
    "apital": "capital", "capitole": "capital", "zimro": "0", "smaller": "a"
}

# Step 1: Replace number words with digits
def replace_number_words(text):
    for word, digit in number_map.items():
        text = re.sub(rf"\b{word}\b", digit, text, flags=re.IGNORECASE)
    return text

# Step 2: Fix common misheard words
def fix_misheard_words(text):
    for wrong, correct in misheard_map.items():
        text = re.sub(rf"\b{wrong}\b", correct, text, flags=re.IGNORECASE)
    return text

# Step 3: Replace "for" with "4"
def replace_for_with_4(text):
    text = re.sub(r"\bfor\b", "4", text, flags=re.IGNORECASE)
    return text

# Step 4: Process capitalization and handle "edge" replacement
def process_capitalization(text):
    matches = re.findall(r"(capital|small) (\w+)", text, re.IGNORECASE)
    cleaned_text = ""

    i = 0
    while i < len(text):
        match_found = False
        
        for match in matches:
            marker, letter = match
            marker_index = text.lower().find(f"{marker.lower()} {letter.lower()}")
            
            if marker_index == i:
                if marker.lower() == "capital":
                    cleaned_text += letter.upper()
                else:
                    cleaned_text += letter.lower()
                i += len(marker) + 2
                match_found = True
                break

        if not match_found:
            cleaned_text += text[i]
            i += 1

    # Handle "edge" replacement based on the previous marker
    cleaned_text = re.sub(r"\bsmall\s+edge\b", "h", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"\bcapital\s+edge\b", "H", cleaned_text, flags=re.IGNORECASE)

    # Remove remaining "capital" or "small" words
    cleaned_text = re.sub(r"\b(capital|small)\b", "", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"[^a-zA-Z0-9]", "", cleaned_text)  # Keep only A-Z, a-z, 0-9
    return cleaned_text

# Function to get Whisper transcription
def get_whisper_transcription(audio_file):
    text = ""
    
    # Try Whisper transcription up to 3 times if length is < 6
    for _ in range(3):
        result = whisper_model.transcribe(audio_file)
        text = result["text"]
        text = replace_number_words(text)  # Replace number words with digits
        text = fix_misheard_words(text)  # Fix misheard words
        text = replace_for_with_4(text)  # Replace "for" with "4"
        text = process_capitalization(text)  # Process capitalization & "edge"
        
        if len(text) >= 6:  # Ensure minimum length of 6
            break

    return text[:6]  # Ensure the output is exactly 6 characters

def get_image_text(image_file):
    ocr_result = ocr_reader.readtext(image_file)
    text = ''.join([res[1] for res in ocr_result])
    return text

def process_all_samples(audio_dir, image_dir):
    all_files = os.listdir(audio_dir)
    total_files = len(all_files)
    exact_match_count = 0  # Counter for exact matches
    
    for i, audio_file in enumerate(tqdm(all_files, desc="Processing")):
        audio_path = os.path.join(audio_dir, audio_file)
        image_file = audio_file.replace('.wav', '.png')  # Assuming matching names
        image_path = os.path.join(image_dir, image_file)

        # Get transcription from Whisper
        whisper_text = get_whisper_transcription(audio_path)
        
        # Get ground truth text from OCR
        ground_truth_text = get_image_text(image_path)

        if not ground_truth_text or not whisper_text:
            continue  

        if whisper_text == ground_truth_text:
            exact_match_count += 1

        if (i + 1) % 500 == 0:
            print(f"\nProgress: {i+1}/{total_files} samples processed. Exact Matches: {exact_match_count}\n")

    # Final count of exact matches
    print(f"\nFinal Exact Matches after processing {total_files} samples: {exact_match_count}\n")

# Run the function to process all samples
process_all_samples(audio_dir, image_dir)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
Processing:  


Progress: 1000/10000 samples processed. Exact Matches: 248



Processing:  15%|█▌        | 1500/10000 [11:07<59:14,  2.39it/s]  


Progress: 1500/10000 samples processed. Exact Matches: 369



Processing:  20%|██        | 2000/10000 [14:58<55:49,  2.39it/s]  

...


Progress: 9000/10000 samples processed. Exact Matches: 2278



Processing:  95%|█████████▌| 9500/10000 [1:10:47<03:00,  2.78it/s]


Progress: 9500/10000 samples processed. Exact Matches: 2401



Processing: 100%|██████████| 10000/10000 [1:14:20<00:00,  2.24it/s]


Progress: 10000/10000 samples processed. Exact Matches: 2542


Final Exact Matches after processing 10000 samples: 2542



# Implementation of Whisper AI(medium) and OCR to generate text for 10 random samples

In [7]:
audio_dir = '/kaggle/input/captcha-dataset/captchas/audio'
image_dir = '/kaggle/input/captcha-dataset/captchas/images'

ocr_reader = Reader(['en'])  # EasyOCR setup
whisper_model = whisper.load_model("medium")  # Whisper model

# Dictionary to convert number words to digits
number_map = {
    "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
    "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9"
}

# Dictionary for common misheard words
misheard_map = {
    "mall": "small", "moll": "small", "capitun": "capital", "capitan": "capital",
    "apital": "capital", "capitole": "capital", "zimro": "0", "smaller": "a"
}

# Step 1: Replace number words with digits
def replace_number_words(text):
    for word, digit in number_map.items():
        text = re.sub(rf"\b{word}\b", digit, text, flags=re.IGNORECASE)
    return text

# Step 2: Fix common misheard words
def fix_misheard_words(text):
    for wrong, correct in misheard_map.items():
        text = re.sub(rf"\b{wrong}\b", correct, text, flags=re.IGNORECASE)
    return text

# Step 3: Replace "for" with "4"
def replace_for_with_4(text):
    text = re.sub(r"\bfor\b", "4", text, flags=re.IGNORECASE)
    return text

# Step 4: Process capitalization and handle "edge" replacement
def process_capitalization(text):
    matches = re.findall(r"(capital|small) (\w+)", text, re.IGNORECASE)
    cleaned_text = ""

    i = 0
    while i < len(text):
        match_found = False
        
        for match in matches:
            marker, letter = match
            marker_index = text.lower().find(f"{marker.lower()} {letter.lower()}")
            
            if marker_index == i:
                if marker.lower() == "capital":
                    cleaned_text += letter.upper()
                else:
                    cleaned_text += letter.lower()
                i += len(marker) + 2
                match_found = True
                break

        if not match_found:
            cleaned_text += text[i]
            i += 1

    # Handle "edge" replacement based on the previous marker
    cleaned_text = re.sub(r"\bsmall\s+edge\b", "h", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"\bcapital\s+edge\b", "H", cleaned_text, flags=re.IGNORECASE)

    # Remove remaining "capital" or "small" words
    cleaned_text = re.sub(r"\b(capital|small)\b", "", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"[^a-zA-Z0-9]", "", cleaned_text)  # Keep only A-Z, a-z, 0-9
    return cleaned_text

# Function to get Whisper transcription
def get_whisper_transcription(audio_file):
    text = ""
    
    # Try Whisper transcription up to 3 times if length is < 6
    for _ in range(3):
        result = whisper_model.transcribe(audio_file)
        text = result["text"]
        text = replace_number_words(text)  # Replace number words with digits
        text = fix_misheard_words(text)  # Fix misheard words
        text = replace_for_with_4(text)  # Replace "for" with "4"
        text = process_capitalization(text)  # Process capitalization & "edge"
        
        if len(text) >= 6:  # Ensure minimum length of 6
            break

    return text[:6]  # Ensure the output is exactly 6 characters

def get_image_text(image_file):
    ocr_result = ocr_reader.readtext(image_file)
    text = ''.join([res[1] for res in ocr_result])
    return text

# Main function to run the process for random 10 files
def process_random_samples(audio_dir, image_dir, num_samples=10):
    audio_files = random.sample(os.listdir(audio_dir), num_samples)
    
    for audio_file in audio_files:
        audio_path = os.path.join(audio_dir, audio_file)
        image_file = audio_file.replace('.wav', '.png')  # Assuming same filename for audio and image
        image_path = os.path.join(image_dir, image_file)

        whisper_text = get_whisper_transcription(audio_path)

        ground_truth_text = get_image_text(image_path)

        print(f"Audio file: {audio_file}")
        print(f"Whisper AI Transcription: {whisper_text}")
        print(f"Ground Truth (OCR): {ground_truth_text}")
        print("="*50)

process_random_samples(audio_dir, image_dir, num_samples=10)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Audio file: captcha_4175.wav
Whisper AI Transcription: mPChvc
Ground Truth (OCR): mPChdc
Audio file: captcha_2383.wav
Whisper AI Transcription: 7X8Fqb
Ground Truth (OCR): 7XBFqb
Audio file: captcha_0550.wav
Whisper AI Transcription: FZQcBF
Ground Truth (OCR): FZQcB4
Audio file: captcha_7525.wav
Whisper AI Transcription: BYVTJD
Ground Truth (OCR): BYVTJD
Audio file: captcha_0435.wav
Whisper AI Transcription: zZWUWQ
Ground Truth (OCR): ZZWUWQ
Audio file: captcha_6497.wav
Whisper AI Transcription: 2BZ51J
Ground Truth (OCR): 2BZ51J
Audio file: captcha_2963.wav
Whisper AI Transcription: w9dKlX
Ground Truth (OCR): wgdKIX
Audio file: captcha_6342.wav
Whisper AI Transcription: RjlwWT
Ground Truth (OCR): RjlwWT
Audio file: captcha_4006.wav
Whisper AI Transcription: JQmbaG
Ground Truth (OCR): JQmbag
Audio file: captcha_3294.wav
Whisper AI Transcription: 5yhjHF
Ground Truth (OCR): SyhjHF


# Whisper AI(medium) Matching Accuracy Scores

In [6]:
audio_dir = '/kaggle/input/captcha-dataset/captchas/audio'
image_dir = '/kaggle/input/captcha-dataset/captchas/images'

ocr_reader = Reader(['en'])  # EasyOCR setup
whisper_model = whisper.load_model("medium")  # Whisper model

# Dictionary to convert number words to digits
number_map = {
    "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
    "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9"
}

# Dictionary for common misheard words
misheard_map = {
    "mall": "small", "moll": "small", "capitun": "capital", "capitan": "capital",
    "apital": "capital", "capitole": "capital", "zimro": "0", "smaller": "a"
}

# Step 1: Replace number words with digits
def replace_number_words(text):
    for word, digit in number_map.items():
        text = re.sub(rf"\b{word}\b", digit, text, flags=re.IGNORECASE)
    return text

# Step 2: Fix common misheard words
def fix_misheard_words(text):
    for wrong, correct in misheard_map.items():
        text = re.sub(rf"\b{wrong}\b", correct, text, flags=re.IGNORECASE)
    return text

# Step 3: Replace "for" with "4"
def replace_for_with_4(text):
    text = re.sub(r"\bfor\b", "4", text, flags=re.IGNORECASE)
    return text

# Step 4: Process capitalization and handle "edge" replacement
def process_capitalization(text):
    matches = re.findall(r"(capital|small) (\w+)", text, re.IGNORECASE)
    cleaned_text = ""

    i = 0
    while i < len(text):
        match_found = False
        
        for match in matches:
            marker, letter = match
            marker_index = text.lower().find(f"{marker.lower()} {letter.lower()}")
            
            if marker_index == i:
                if marker.lower() == "capital":
                    cleaned_text += letter.upper()
                else:
                    cleaned_text += letter.lower()
                i += len(marker) + 2
                match_found = True
                break

        if not match_found:
            cleaned_text += text[i]
            i += 1

    # Handle "edge" replacement based on the previous marker
    cleaned_text = re.sub(r"\bsmall\s+edge\b", "h", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"\bcapital\s+edge\b", "H", cleaned_text, flags=re.IGNORECASE)

    # Remove remaining "capital" or "small" words
    cleaned_text = re.sub(r"\b(capital|small)\b", "", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"[^a-zA-Z0-9]", "", cleaned_text)  # Keep only A-Z, a-z, 0-9
    return cleaned_text

# Function to get Whisper transcription
def get_whisper_transcription(audio_file):
    text = ""
    
    # Try Whisper transcription up to 3 times if length is < 6
    for _ in range(3):
        result = whisper_model.transcribe(audio_file)
        text = result["text"]
        text = replace_number_words(text)  # Replace number words with digits
        text = fix_misheard_words(text)  # Fix misheard words
        text = replace_for_with_4(text)  # Replace "for" with "4"
        text = process_capitalization(text)  # Process capitalization & "edge"
        
        if len(text) >= 6:  # Ensure minimum length of 6
            break

    return text[:6]  # Ensure the output is exactly 6 characters

def get_image_text(image_file):
    ocr_result = ocr_reader.readtext(image_file)
    text = ''.join([res[1] for res in ocr_result])
    return text

def process_all_samples(audio_dir, image_dir):
    all_files = os.listdir(audio_dir)
    total_files = len(all_files)
    cer_scores = []
    
    for i, audio_file in enumerate(tqdm(all_files, desc="Processing")):
        audio_path = os.path.join(audio_dir, audio_file)
        image_file = audio_file.replace('.wav', '.png')  # Assuming matching names
        image_path = os.path.join(image_dir, image_file)

        whisper_text = get_whisper_transcription(audio_path)

        ground_truth_text = get_image_text(image_path)

        if not ground_truth_text or not whisper_text:
            continue  

        error_rate = cer(ground_truth_text, whisper_text)  # CER calculation
        accuracy = max(0, 100 - (error_rate * 100))  # Convert to percentage accuracy
        cer_scores.append(accuracy)

        # Print progress every 500 samples
        if (i + 1) % 500 == 0:
            avg_accuracy = sum(cer_scores) / len(cer_scores)
            print(f"\nProgress: {i+1}/{total_files} samples processed. Current Accuracy: {avg_accuracy:.2f}%\n")

    avg_accuracy = sum(cer_scores) / len(cer_scores) if cer_scores else 0
    print(f"\nFinal Accuracy after processing {total_files} samples: {avg_accuracy:.2f}%\n")

# Run the function to process all samples
process_all_samples(audio_dir, image_dir)

100%|██████████████████████████████████████| 1.42G/1.42G [00:07<00:00, 195MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper


Progress: 500/10000 samples processed. Current Accuracy: 79.25%



Processing:  10%|█         | 1000/10000 [20:49<2:52:04,  1.15s/it]


Progress: 1000/10000 samples processed. Current Accuracy: 79.66%



Processing:  15%|█▌        | 1500/10000 [31:28<2:47:09,  1.18s/it]

...


Progress: 4500/10000 samples processed. Current Accuracy: 79.46%



Processing:  50%|█████     | 5000/10000 [1:45:14<1:40:58,  1.21s/it]


Progress: 5000/10000 samples processed. Current Accuracy: 79.53%



Processing:  50%|█████     | 5035/10000 [1:45:56<1:44:27,  1.26s/it]


KeyboardInterrupt: 

# Whisper AI(medium) - No. of texts matched

In [10]:
audio_dir = '/kaggle/input/captcha-dataset/captchas/audio'
image_dir = '/kaggle/input/captcha-dataset/captchas/images'

ocr_reader = Reader(['en'])  # EasyOCR setup
whisper_model = whisper.load_model("medium")  # Whisper model

# Dictionary to convert number words to digits
number_map = {
    "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
    "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9"
}

# Dictionary for common misheard words
misheard_map = {
    "mall": "small", "moll": "small", "capitun": "capital", "capitan": "capital",
    "apital": "capital", "capitole": "capital", "zimro": "0", "smaller": "a"
}

# Step 1: Replace number words with digits
def replace_number_words(text):
    for word, digit in number_map.items():
        text = re.sub(rf"\b{word}\b", digit, text, flags=re.IGNORECASE)
    return text

# Step 2: Fix common misheard words
def fix_misheard_words(text):
    for wrong, correct in misheard_map.items():
        text = re.sub(rf"\b{wrong}\b", correct, text, flags=re.IGNORECASE)
    return text

# Step 3: Replace "for" with "4"
def replace_for_with_4(text):
    text = re.sub(r"\bfor\b", "4", text, flags=re.IGNORECASE)
    return text

# Step 4: Process capitalization and handle "edge" replacement
def process_capitalization(text):
    matches = re.findall(r"(capital|small) (\w+)", text, re.IGNORECASE)
    cleaned_text = ""

    i = 0
    while i < len(text):
        match_found = False
        
        for match in matches:
            marker, letter = match
            marker_index = text.lower().find(f"{marker.lower()} {letter.lower()}")
            
            if marker_index == i:
                if marker.lower() == "capital":
                    cleaned_text += letter.upper()
                else:
                    cleaned_text += letter.lower()
                i += len(marker) + 2
                match_found = True
                break

        if not match_found:
            cleaned_text += text[i]
            i += 1

    # Handle "edge" replacement based on the previous marker
    cleaned_text = re.sub(r"\bsmall\s+edge\b", "h", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"\bcapital\s+edge\b", "H", cleaned_text, flags=re.IGNORECASE)

    # Remove remaining "capital" or "small" words
    cleaned_text = re.sub(r"\b(capital|small)\b", "", cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r"[^a-zA-Z0-9]", "", cleaned_text)  # Keep only A-Z, a-z, 0-9
    return cleaned_text

# Function to get Whisper transcription
def get_whisper_transcription(audio_file):
    text = ""
    
    # Try Whisper transcription up to 3 times if length is < 6
    for _ in range(3):
        result = whisper_model.transcribe(audio_file)
        text = result["text"]
        text = replace_number_words(text)  # Replace number words with digits
        text = fix_misheard_words(text)  # Fix misheard words
        text = replace_for_with_4(text)  # Replace "for" with "4"
        text = process_capitalization(text)  # Process capitalization & "edge"
        
        if len(text) >= 6:  # Ensure minimum length of 6
            break

    return text[:6]  # Ensure the output is exactly 6 characters

def get_image_text(image_file):
    ocr_result = ocr_reader.readtext(image_file)
    text = ''.join([res[1] for res in ocr_result])
    return text

def process_all_samples(audio_dir, image_dir):
    all_files = os.listdir(audio_dir)
    total_files = len(all_files)
    exact_match_count = 0  # Counter for exact matches
    
    for i, audio_file in enumerate(tqdm(all_files, desc="Processing")):
        audio_path = os.path.join(audio_dir, audio_file)
        image_file = audio_file.replace('.wav', '.png')  # Assuming matching names
        image_path = os.path.join(image_dir, image_file)

        whisper_text = get_whisper_transcription(audio_path)

        ground_truth_text = get_image_text(image_path)

        if not ground_truth_text or not whisper_text:
            continue  

        if whisper_text == ground_truth_text:
            exact_match_count += 1

        if (i + 1) % 500 == 0:
            print(f"\nProgress: {i+1}/{total_files} samples processed. Exact Matches: {exact_match_count}\n")

    # Final count of exact matches
    print(f"\nFinal Exact Matches after processing {total_files} samples: {exact_match_count}\n")

# Run the function to process all samples
process_all_samples(audio_dir, image_dir)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
Processing:  


Progress: 500/10000 samples processed. Exact Matches: 151



Processing:  10%|█         | 1000/10000 [20:31<2:51:11,  1.14s/it]


Progress: 1000/10000 samples processed. Exact Matches: 296



Processing:  15%|█▌        | 1500/10000 [31:01<2:45:27,  1.17s/it]

...


Progress: 9000/10000 samples processed. Exact Matches: 2668



Processing:  95%|█████████▌| 9500/10000 [3:18:12<09:45,  1.17s/it]


Progress: 9500/10000 samples processed. Exact Matches: 2808



Processing: 100%|██████████| 10000/10000 [3:28:28<00:00,  1.25s/it]


Progress: 10000/10000 samples processed. Exact Matches: 2962


Final Exact Matches after processing 10000 samples: 2962

